In [1]:
import pandas as pd
from geopy.geocoders import Photon
from geopy.geocoders import Nominatim
from unidecode import unidecode
import unidecode


In [2]:
import geopandas as gpd

In [3]:
import numpy as np
from shapely.geometry import Point, Polygon

In [ ]:
pip install --upgrade --force-reinstall rtree


In [ ]:
!pip install geopandas


In [ ]:
!pip install geopandas

In [24]:
# Cargar el archivo CSV
datos_csv = pd.read_csv('/Users/pablouriarte/Documents/1. Expediente Tec de Monterrey/1.Tesis/Mapa_Migracion_Irregular_Mexico/3. mapas/1. Mapas/Metodo_1/ciudades_rutas/ciudades detenciones/ciudades_detenciones.csv')


In [25]:
# Cambiar los nombres de las columnas 'Unnamed: 0' y 'Unnamed: 1'
datos_csv = datos_csv.rename(columns={'Unnamed: 0': 'ciudad', 'Unnamed: 1': 'detenciones'})

datos_csv = datos_csv.dropna()

# Verificar los cambios mostrando las primeras filas del DataFrame
print(datos_csv.head())


                            ciudad detenciones
0                   Aguascalientes         559
1             Aguascalientes, Ags.         423
2                      Cosío, Ags.          98
3                Jesús María, Ags.          20
4  San Francisco de los Romo, Ags.          18


In [27]:
# Inicializamos la columna 'estado' con NaN
datos_csv['estado'] = pd.NA

# Almacenamos temporalmente la etiqueta de estado actual
etiqueta_estado_actual = pd.NA

# Paso 2: Asignar la etiqueta de estado a la nueva columna 'estado'
for idx, tiene_etiqueta in es_etiqueta_estado.items():
    if tiene_etiqueta:  # Si la fila actual es una etiqueta de estado
        etiqueta_estado_actual = datos_csv.at[idx, 'ciudad']  # Actualiza la etiqueta de estado
    datos_csv.at[idx, 'estado'] = etiqueta_estado_actual  # Asigna la etiqueta de estado actual a la columna 'estado'

# Opcionalmente, puedes querer limpiar los valores de 'estado' para quitar la parte ', Ags.' o similar
datos_csv['estado'] = datos_csv['estado'].str.replace(',.*', '', regex=True)

# Verificación: Muestra las primeras filas para ver el resultado
print(datos_csv.head())

TypeError: boolean value of NA is ambiguous

In [13]:
# Crear un filtro para identificar filas donde 'ciudad' contiene ',' o '.'
filtro = datos_csv['ciudad'].str.contains(',', regex=False) | datos_csv['ciudad'].str.contains('\.', regex=True)

# Aplicar el filtro para mantener solo las filas que cumplen con la condición
datos_csv_filtrado = datos_csv[filtro]



In [15]:
datos_csv.head()

,ciudad,detenciones
0,Aguascalientes,559
1,"Aguascalientes, Ags.",423
2,"Cosío, Ags.",98
3,"Jesús María, Ags.",20
4,"San Francisco de los Romo, Ags.",18


In [18]:
# Inicializamos la columna 'estado' con NaN
datos_csv['estado'] = pd.NA

# Almacenamos temporalmente la etiqueta de estado actual
etiqueta_estado_actual = pd.NA




In [ ]:
# Primero, separa 'ciudad' por la coma y toma solo la parte antes de la coma
datos_csv['ciudad'] = datos_csv['ciudad'].str.split(',').str[0]

# Luego, concatena esa parte con el valor correspondiente de 'estado'
datos_csv['ciudad'] = datos_csv['ciudad'] + ', ' + datos_csv['estado']

# Verificación: Muestra las primeras filas para ver el resultado
print(datos_csv.head())

In [23]:
# Suponiendo que estamos interesados en filas donde 'ciudad' no contiene una coma ni un punto
es_etiqueta_estado = ~datos_csv['ciudad'].str.contains(',|\.')

# Paso 2: Asignar la etiqueta de estado a la nueva columna 'estado'
for idx, tiene_etiqueta in es_etiqueta_estado.items():
    if tiene_etiqueta:  # Si la fila actual es una etiqueta de estado
        etiqueta_estado_actual = datos_csv.at[idx, 'ciudad']  # Actualiza la etiqueta de estado
    datos_csv.at[idx, 'estado'] = etiqueta_estado_actual  # Asigna la etiqueta de estado actual a la columna 'estado'

# Opcionalmente, puedes querer limpiar los valores de 'estado' para quitar la parte ', Ags.' o similar
datos_csv['estado'] = datos_csv['estado'].str.replace(',.*', '', regex=True)

# Verificación: Muestra las primeras filas para ver el resultado
print(datos_csv.head(10))

TypeError: boolean value of NA is ambiguous

  ciudad detenciones estado
0   <NA>         559   <NA>
1   <NA>         423   <NA>
2   <NA>          98   <NA>
3   <NA>          20   <NA>
4   <NA>          18   <NA>


In [ ]:
# Cambiar las etiquetas de 'Mexico' a 'Estado de Mexico' en la columna 'estado'
datos_csv['estado'] = datos_csv['estado'].replace('Mexico', 'Estado de Mexico')

In [ ]:
datos_csv.shape

In [ ]:
import pandas as pd
from unidecode import unidecode

# Corrección en la importación de unidecode
def quitar_acentos(valor):
    if isinstance(valor, str):  # Solo aplica a cadenas de texto
        return unidecode(valor)  # Uso correcto de unidecode
    return valor

# Aplicar la función quitar_acentos a todas las columnas del DataFrame
for columna in datos_csv.columns:
    datos_csv[columna] = datos_csv[columna].apply(quitar_acentos)

# Verificación: Mostrar las primeras filas para asegurarse de que los acentos han sido quitados
print(datos_csv.head())


In [ ]:
# Definir las listas de estados para cada ruta, asegurándose de que todas las entradas están correctamente formateadas como strings
estados_ruta_golfo = ['Tabasco', 'Nuevo Leon', 'Veracruz', 'Tamaulipas']
estados_ruta_pacifico = ['Guerrero', 'Michoacan', 'Jalisco', 'Nayarit', 'Sinaloa', 'Sonora', 'Baja California', 'Baja California Sur','Colima']
estados_ruta_centro = ['San Luis Potosi','Estado de Mexico', 'Tlaxcala', 'Ciudad de Mexico', 'Puebla', 'Morelos', 'Hidalgo', 'Zacatecas', 'Durango', 'Chihuahua', 'Coahuila', 'Queretaro', 'Aguascalientes', 'Guanajuato']
ruta_sur = ['Chiapas', 'Oaxaca', 'Tabasco', 'Campeche', 'Quintana Roo', 'Yucatan']

# Actualizar la función para asignar la ruta basada en el estado
def asignar_ruta(estado):
    estado = estado.strip()  # Eliminar espacios en blanco al principio y al final
    if estado in estados_ruta_golfo:
        return 'Ruta del Golfo'
    elif estado in estados_ruta_pacifico:
        return 'Ruta del Pacifico'
    elif estado in estados_ruta_centro:
        return 'Ruta Centro'
    elif estado in ruta_sur:
        return 'Ruta Sur'
    else:
        return 'Ruta no identificada'  # Para estados que no coinciden con las listas

# Aplicar la función al DataFrame para crear una nueva columna 'Ruta'
datos_csv['ruta'] = datos_csv['estado'].apply(asignar_ruta)

In [ ]:
# Reordenar las columnas para colocar 'Estado' después de 'Ciudad'
column_order = ['ciudad', 'estado', 'ruta','detenciones']

# Aplicar el nuevo orden de columnas al DataFrame
datos_csv = datos_csv[column_order]

In [ ]:
# Obtenemos los valores únicos de la columna 'ruta'
valores_unicos_ruta = datos_csv['ruta'].unique()

# Imprimimos los valores únicos
print(valores_unicos_ruta)

In [ ]:
# Initialize the geolocator using OSM Nominatim
geolocator = Photon(user_agent="measurement", timeout=10)

# Define a function to get latitude and longitude
def get_lat_lon(address):
    try:
        location = geolocator.geocode(address)
        if location:
            return location.latitude, location.longitude
        else:
            return '', ''  # Return empty strings if location is not found
    except AttributeError:
        return '', ''

# Apply the function to the 'City' column and split the results into 'Latitude' and 'Longitude' columns
datos_csv[['latitud', 'longitud']] = datos_csv['ciudad'].apply(lambda x: pd.Series(get_lat_lon(x)))

In [ ]:
# Verificar los cambios mostrando las primeras filas del DataFrame filtrado
print(datos_csv.head(20))

In [ ]:
# Especifica la ruta donde deseas guardar el archivo CSV
ruta_archivo_csv = '/Users/pablouriarte/Documents/1. Expediente Tec de Monterrey/1.Tesis/Mapa_Migracion_Irregular_Mexico/3. mapas/1. Mapas/Metodo_1/ciudades_rutas/ciudades detenciones/ciudades_detenciones_1.csv'

# Guardar el DataFrame en el archivo CSV especificado
datos_csv.to_csv(ruta_archivo_csv, index=False)

print(f"El DataFrame ha sido guardado exitosamente en: {ruta_archivo_csv}")


In [132]:
# Cargar el archivo CSV
datos_csv1 = pd.read_csv('/Users/pablouriarte/Documents/1. Expediente Tec de Monterrey/1.Tesis/Mapa_Migracion_Irregular_Mexico/3. mapas/1. Mapas/Metodo_1/ciudades_rutas/ciudades detenciones/ciudades_detenciones_1.csv')


In [133]:
datos_csv1.head(29)

,ciudad,estado,ruta,detenciones,latitud,longitud,tipo,ID_Poligono,geometry,codigo_ruta,contador_xy,codigo_unico
0,"Ensenada, Baja California",Baja California,Ruta del Pacifico,29,31.865889,-116.602983,Presencia Migración Irregular,PG104,"POLYGON ((-117 32, -116 32, -116 31, -117 31, ...",P,1,P-PG104-1
1,"Playas de Rosarito, Baja California",Baja California,Ruta del Pacifico,38,32.258129,-116.873307,Presencia Migración Irregular,PG105,"POLYGON ((-117 33, -116 33, -116 32, -117 32, ...",P,1,P-PG105-1
2,"Tecate, Baja California",Baja California,Ruta del Pacifico,594,32.565383,-116.629879,Presencia Migración Irregular,PG105,"POLYGON ((-117 33, -116 33, -116 32, -117 32, ...",P,2,P-PG105-2
3,"Mexicali, Baja California",Baja California,Ruta del Pacifico,0,32.640525,-115.474899,Presencia Migración Irregular,PG132,"POLYGON ((-116 33, -115 33, -115 32, -116 32, ...",P,1,P-PG132-1
4,"San Felipe, Baja California",Baja California,Ruta del Pacifico,4,31.026953,-114.839607,Presencia Migración Irregular,PG158,"POLYGON ((-115 32, -114 32, -114 31, -115 31, ...",P,1,P-PG158-1
5,"San Luis Rio Colorado, Sonora",Sonora,Ruta del Pacifico,67,32.451796,-114.765254,Presencia Migración Irregular,PG159,"POLYGON ((-115 33, -114 33, -114 32, -115 32, ...",P,1,P-PG159-1
6,"Puerto Penasco, Sonora",Sonora,Ruta del Pacifico,1,31.317087,-113.531819,Presencia Migración Irregular,PG185,"POLYGON ((-114 32, -113 32, -113 31, -114 31, ...",P,1,P-PG185-1
7,"Caborca, Sonora",Sonora,Ruta del Pacifico,414,30.716601,-112.156260,Presencia Migración Irregular,PG211,"POLYGON ((-113 31, -112 31, -112 30, -113 30, ...",P,1,P-PG211-1
8,"General Plutarco Elias Calles, Sonora",Sonora,Ruta del Pacifico,0,31.694984,-112.711749,Presencia Migración Irregular,PG212,"POLYGON ((-113 32, -112 32, -112 31, -113 31, ...",P,1,P-PG212-1
9,"Santa Ana, Sonora",Sonora,Ruta del Pacifico,0,30.541092,-111.118488,Presencia Migración Irregular,PG238,"POLYGON ((-112 31, -111 31, -111 30, -112 30, ...",P,1,P-PG238-1


In [ ]:
# Convertir la columna 'detenciones' a entero
datos_csv1['detenciones'] = pd.to_numeric(datos_csv1['detenciones'], errors='coerce').fillna(0).astype(int)

In [ ]:
# Identificar los índices de la primera fila de cada categoría en 'estado'
indices_a_eliminar = datos_csv1.groupby('estado').head(1).index

# Eliminar esas filas del DataFrame
datos_csv1 = datos_csv1.drop(indices_a_eliminar)




In [ ]:
# Verificar los cambios
print(datos_csv1.head())


In [ ]:
# Agregar la columna 'tipo' con la etiqueta 'Presencia Migración Irregular' para todas las filas
datos_csv1['tipo'] = 'Presencia Migración Irregular'

# Verificar los cambios mostrando las primeras filas del DataFrame
print(datos_csv1.head())


In [20]:
# Especifica la ruta donde deseas guardar el archivo CSV
ruta_archivo_csv1 = '/Users/pablouriarte/Documents/1. Expediente Tec de Monterrey/1.Tesis/Mapa_Migracion_Irregular_Mexico/3. mapas/1. Mapas/Metodo_1/ciudades_rutas/ciudades detenciones/ciudades_detenciones_1.csv'

# Guardar el DataFrame en el archivo CSV especificado
datos_csv1.to_csv(ruta_archivo_csv1, index=False)

print(f"El DataFrame ha sido guardado exitosamente en: {ruta_archivo_csv1}")


NameError: name 'datos_csv1' is not defined

### Grid Shape

Para crear un grid con polígonos geoespaciales y asignar a cada punto según su latitud y longitud a un polígono correspondiente, se puede utilizar la biblioteca geopandas en Python, que extiende las capacidades de pandas para trabajar con datos espaciales. También usaremos shapely para manipular y analizar datos geométricos planos, y numpy para manejar operaciones numéricas.



Crear el Grid de Polígonos: Primero, generamos un conjunto de polígonos que representen cada celda del grid sobre el área de interés (en este caso, México). Cada polígono puede ser identificado de manera única, por ejemplo, usando una convención de nomenclatura como PG1, PG2, etc.

In [96]:
def crear_grid(xmin, xmax, ymin, ymax, ancho_celda, alto_celda):
    cols = list(np.arange(xmin, xmax, ancho_celda))
    rows = list(np.arange(ymin, ymax, alto_celda))
    poligonos = []
    ids = []

    id_contador = 1  # Iniciar contador para los ID de polígonos
    for x in cols:
        for y in rows:
            poligonos.append(Polygon([(x, y), (x+ancho_celda, y), (x+ancho_celda, y-alto_celda), (x, y-alto_celda)]))
            ids.append(f"PG{id_contador}")  # Asignar un ID único a cada polígono
            id_contador += 1

    grid = gpd.GeoDataFrame({'ID_Poligono': ids, 'geometry': poligonos})
    return grid

# Generar el grid con los límites aproximados de México y celdas de 1 grado por 1 grado
xmin, ymin, xmax, ymax = -120, 10, -82, 37
grid = crear_grid(xmin, xmax, ymin, ymax, 1, 1)

# Ahora, 'grid' es un GeoDataFrame que contiene el 'ID_Poligono' y su 'geometry'
print(grid.head())

  ID_Poligono                                           geometry
0         PG1  POLYGON ((-120.00000 10.00000, -119.00000 10.0...
1         PG2  POLYGON ((-120.00000 11.00000, -119.00000 11.0...
2         PG3  POLYGON ((-120.00000 12.00000, -119.00000 12.0...
3         PG4  POLYGON ((-120.00000 13.00000, -119.00000 13.0...
4         PG5  POLYGON ((-120.00000 14.00000, -119.00000 14.0...


In [134]:
# Cargar el archivo CSV
datos_csv1 = pd.read_csv('/Users/pablouriarte/Documents/1. Expediente Tec de Monterrey/1.Tesis/Mapa_Migracion_Irregular_Mexico/3. mapas/1. Mapas/Metodo_1/ciudades_rutas/ciudades detenciones/ciudades_detenciones_1.csv')


In [135]:
# Actualizar Campeche, Campeche
datos_csv1.loc[(datos_csv1['ciudad'] == 'Campeche, Campeche'), 'latitud'] = 19.830285751988043
datos_csv1.loc[(datos_csv1['ciudad'] == 'Campeche, Campeche'), 'longitud'] = -90.53551463998544

# Actualizar Colima, Colima
datos_csv1.loc[(datos_csv1['ciudad'] == 'Colima, Colima'), 'latitud'] = 19.24487815537049
datos_csv1.loc[(datos_csv1['ciudad'] == 'Colima, Colima'), 'longitud'] = -103.7250306766154

# Actualizar Puebla, Puebla
datos_csv1.loc[(datos_csv1['ciudad'] == 'Puebla, Puebla'), 'latitud'] = 19.038406227976886
datos_csv1.loc[(datos_csv1['ciudad'] == 'Puebla, Puebla'), 'longitud'] = -98.2069491554557


datos_csv1.loc[(datos_csv1['ciudad'] == 'Zacatecas, Zacatecas'), 'latitud'] = 19.039018782328817
datos_csv1.loc[(datos_csv1['ciudad'] == 'Zacatecas, Zacatecas'), 'longitud'] = -98.20248616428334

In [136]:
datos_csv1.head(30)

,ciudad,estado,ruta,detenciones,latitud,longitud,tipo,ID_Poligono,geometry,codigo_ruta,contador_xy,codigo_unico
0,"Ensenada, Baja California",Baja California,Ruta del Pacifico,29,31.865889,-116.602983,Presencia Migración Irregular,PG104,"POLYGON ((-117 32, -116 32, -116 31, -117 31, ...",P,1,P-PG104-1
1,"Playas de Rosarito, Baja California",Baja California,Ruta del Pacifico,38,32.258129,-116.873307,Presencia Migración Irregular,PG105,"POLYGON ((-117 33, -116 33, -116 32, -117 32, ...",P,1,P-PG105-1
2,"Tecate, Baja California",Baja California,Ruta del Pacifico,594,32.565383,-116.629879,Presencia Migración Irregular,PG105,"POLYGON ((-117 33, -116 33, -116 32, -117 32, ...",P,2,P-PG105-2
3,"Mexicali, Baja California",Baja California,Ruta del Pacifico,0,32.640525,-115.474899,Presencia Migración Irregular,PG132,"POLYGON ((-116 33, -115 33, -115 32, -116 32, ...",P,1,P-PG132-1
4,"San Felipe, Baja California",Baja California,Ruta del Pacifico,4,31.026953,-114.839607,Presencia Migración Irregular,PG158,"POLYGON ((-115 32, -114 32, -114 31, -115 31, ...",P,1,P-PG158-1
5,"San Luis Rio Colorado, Sonora",Sonora,Ruta del Pacifico,67,32.451796,-114.765254,Presencia Migración Irregular,PG159,"POLYGON ((-115 33, -114 33, -114 32, -115 32, ...",P,1,P-PG159-1
6,"Puerto Penasco, Sonora",Sonora,Ruta del Pacifico,1,31.317087,-113.531819,Presencia Migración Irregular,PG185,"POLYGON ((-114 32, -113 32, -113 31, -114 31, ...",P,1,P-PG185-1
7,"Caborca, Sonora",Sonora,Ruta del Pacifico,414,30.716601,-112.156260,Presencia Migración Irregular,PG211,"POLYGON ((-113 31, -112 31, -112 30, -113 30, ...",P,1,P-PG211-1
8,"General Plutarco Elias Calles, Sonora",Sonora,Ruta del Pacifico,0,31.694984,-112.711749,Presencia Migración Irregular,PG212,"POLYGON ((-113 32, -112 32, -112 31, -113 31, ...",P,1,P-PG212-1
9,"Santa Ana, Sonora",Sonora,Ruta del Pacifico,0,30.541092,-111.118488,Presencia Migración Irregular,PG238,"POLYGON ((-112 31, -111 31, -111 30, -112 30, ...",P,1,P-PG238-1


In [127]:
# Buscar filas con NaN en cualquier columna
filas_con_nan = datos_csv1[datos_csv1.isna().any(axis=1)]
filas_con_nan


,ciudad,estado,ruta,detenciones,latitud,longitud,tipo,ID_Poligono,geometry,codigo_ruta,contador_xy,codigo_unico


In [128]:
# Eliminar las columnas del DataFrame
#datos_csv1 = datos_csv1.drop(columns=['ID_Poligono','geometry', 'codigo_ruta', 'geometry_y', 'geometry'])

# Verificación: Muestra las primeras filas para asegurarse de que las columnas han sido eliminadas
#print(datos_csv1.head())


KeyError: "['ID', 'geometry_x', 'geometry_y'] not found in axis"

In [130]:
# Convertir la columna 'detenciones' a entero
datos_csv1['detenciones'] = pd.to_numeric(datos_csv1['detenciones'], errors='coerce').fillna(0).astype(int)

In [137]:
# Paso 2: Asignar Puntos a Polígonos
puntos = [Point(xy) for xy in zip(datos_csv1.longitud, datos_csv1.latitud)]
geo_df = gpd.GeoDataFrame(datos_csv1, geometry=puntos)

# Realizar la asignación
geo_df = gpd.sjoin(geo_df, grid, how='left', op='within')

# Paso 3: Actualizar DataFrame
datos_csv1['ID_Poligono'] = geo_df['ID_Poligono']

/Users/pablouriarte/opt/anaconda3/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3377: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if (await self.run_code(code, result,  async_=asy)):


KeyError: 'ID_Poligono'

In [109]:
# Asegurándose de que datos_csv2 es un GeoDataFrame
if not isinstance(datos_csv1, gpd.GeoDataFrame):
    # Si datos_csv2 ya tiene una columna 'geometry', úsala; de lo contrario, se necesita definir una
    if 'geometry' in datos_csv1.columns:
        datos_csv1 = gpd.GeoDataFrame(datos_csv1, geometry='geometry')
    else:
        # Aquí se asume que no hay una columna 'geometry' existente y que quizás no necesites definir una en este momento
        datos_csv1 = gpd.GeoDataFrame(datos_csv1)

# Realizar el merge (unión) entre datos_csv2 y grid usando 'ID_Poligono' como clave
datos_csv1 = datos_csv1.merge(grid[['ID_Poligono', 'geometry']], on='ID_Poligono', how='left')


In [115]:
datos_csv1.tail(20)

,ciudad,estado,ruta,detenciones,latitud,longitud,tipo,ID_Poligono,geometry,codigo_ruta,contador_xy,codigo_unico
269,"Balancan, Tabasco",Tabasco,Ruta del Golfo,0,17.797517,-91.345859,Presencia Migración Irregular,PG765,"POLYGON ((-92.00000 18.00000, -91.00000 18.000...",G,4,G-PG765-4
15,"Carmen, Campeche",Campeche,Ruta Sur,255,18.651739,-91.814920,Presencia Migración Irregular,PG766,"POLYGON ((-92.00000 19.00000, -91.00000 19.000...",S,1,S-PG766-1
9,"Tijuana, Baja California",Baja California,Ruta del Pacifico,0,32.531740,-117.019529,Presencia Migración Irregular,PG78,"POLYGON ((-118.00000 33.00000, -117.00000 33.0...",P,1,P-PG78-1
17,"Escarcega, Campeche",Campeche,Ruta Sur,0,18.606934,-90.738582,Presencia Migración Irregular,PG793,"POLYGON ((-91.00000 19.00000, -90.00000 19.000...",S,1,S-PG793-1
16,"Champoton, Campeche",Campeche,Ruta Sur,150,19.343965,-90.724274,Presencia Migración Irregular,PG794,"POLYGON ((-91.00000 20.00000, -90.00000 20.000...",S,1,S-PG794-1
19,"Seybaplaya, Campeche",Campeche,Ruta Sur,8,19.603372,-90.625367,Presencia Migración Irregular,PG794,"POLYGON ((-91.00000 20.00000, -90.00000 20.000...",S,2,S-PG794-2
14,"Campeche, Campeche",Campeche,Ruta Sur,0,19.830286,-90.535515,Presencia Migración Irregular,PG794,"POLYGON ((-91.00000 20.00000, -90.00000 20.000...",S,3,S-PG794-3
20,"Tenabo, Campeche",Campeche,Ruta Sur,22,20.040661,-90.225496,Presencia Migración Irregular,PG795,"POLYGON ((-91.00000 21.00000, -90.00000 21.000...",S,1,S-PG795-1
13,"Calkini, Campeche",Campeche,Ruta Sur,1,20.369512,-90.051078,Presencia Migración Irregular,PG795,"POLYGON ((-91.00000 21.00000, -90.00000 21.000...",S,2,S-PG795-2
12,"Calakmul, Campeche",Campeche,Ruta Sur,194,18.107341,-89.808328,Presencia Migración Irregular,PG820,"POLYGON ((-90.00000 19.00000, -89.00000 19.000...",S,1,S-PG820-1


Asignar Puntos a Polígonos: Luego, para cada punto dado por sus coordenadas de latitud y longitud en datos_csv1, determinamos en qué polígono del grid se encuentra y asignamos el identificador de ese polígono al punto.


el codigo de un punto se miraria algo asi: S-PG#-1

Para asignar un código que combine el código de ruta por estado, el identificador del polígono al que pertenece cada punto y un número ascendente basado en el orden de las coordenadas XY dentro de cada polígono

por estado, si:
    'Ruta Sur': 'S',
    'Ruta del Pacifico': 'P',
    'Ruta Centro': 'C',
    'Ruta del Golfo': 'G'
    
por ID_Poligono: el poligono a que pertenece

y por orden asendientes de su coordenada xy dentro de cada poligono

In [113]:
# Primero, mapeamos la columna 'ruta' a su código correspondiente
ruta_to_code = {
    'Ruta Sur': 'S',
    'Ruta del Pacifico': 'P',
    'Ruta Centro': 'C',
    'Ruta del Golfo': 'G'
}
datos_csv1['codigo_ruta'] = datos_csv1['ruta'].map(ruta_to_code)

# Asumiendo que 'datos_csv2' ya tiene una columna 'ID_Poligono',
# vamos a crear una columna que identifique el orden XY dentro de cada polígono
# Para esto, primero ordenamos el DataFrame
datos_csv1.sort_values(by=['ID_Poligono', 'latitud', 'longitud'], ascending=[True, True, True], inplace=True)

# Creamos una nueva columna que será el contador dentro de cada grupo de 'ID_Poligono'
datos_csv1['contador_xy'] = datos_csv1.groupby('ID_Poligono').cumcount() + 1

# Ahora, combinamos todo para crear el código único
datos_csv1['codigo_unico'] = datos_csv1['codigo_ruta'] + '-' + datos_csv1['ID_Poligono'] + '-' + datos_csv1['contador_xy'].astype(str)

# Revisamos el resultado
print(datos_csv1[['ciudad', 'estado', 'ruta', 'ID_Poligono', 'codigo_unico']])


                                  ciudad           estado               ruta  \
4              Ensenada, Baja California  Baja California  Ruta del Pacifico   
6    Playas de Rosarito, Baja California  Baja California  Ruta del Pacifico   
8                Tecate, Baja California  Baja California  Ruta del Pacifico   
5              Mexicali, Baja California  Baja California  Ruta del Pacifico   
7            San Felipe, Baja California  Baja California  Ruta del Pacifico   
..                                   ...              ...                ...   
247                  Tulum, Quintana Roo     Quintana Roo           Ruta Sur   
246            Solidaridad, Quintana Roo     Quintana Roo           Ruta Sur   
243                Cozumel, Quintana Roo     Quintana Roo           Ruta Sur   
242          Benito Juarez, Quintana Roo     Quintana Roo           Ruta Sur   
244           Isla Mujeres, Quintana Roo     Quintana Roo           Ruta Sur   

    ID_Poligono codigo_unico  
4       

Actualizar DataFrame: Finalmente, agregamos una nueva columna al DataFrame datos_csv1 para almacenar el identificador del polígono del grid al que pertenece cada punto.

In [117]:
# Obtener el conteo de los valores únicos en la columna 'ID_Poligono'
conteo_id_poligono = datos_csv1['ID_Poligono'].value_counts()

# Mostrar el conteo
print(conteo_id_poligono)



ID_Poligono
PG578    37
PG551    19
PG630    15
PG530    15
PG631    12
         ..
PG450     1
PG446     1
PG426     1
PG421     1
PG554     1
Name: count, Length: 121, dtype: int64


In [119]:
# Especifica la ruta donde deseas guardar el archivo CSV
ruta_archivo_csv3 = '/Users/pablouriarte/Documents/1. Expediente Tec de Monterrey/1.Tesis/Mapa_Migracion_Irregular_Mexico/3. mapas/1. Mapas/Metodo_1/ciudades_rutas/ciudades detenciones/ciudades_detenciones_1.csv'

# Guardar el DataFrame en el archivo CSV especificado
datos_csv1.to_csv(ruta_archivo_csv3, index=False)

print(f"El DataFrame ha sido guardado exitosamente en: {ruta_archivo_csv3}")

El DataFrame ha sido guardado exitosamente en: /Users/pablouriarte/Documents/1. Expediente Tec de Monterrey/1.Tesis/Mapa_Migracion_Irregular_Mexico/3. mapas/1. Mapas/Metodo_1/ciudades_rutas/ciudades detenciones/ciudades_detenciones_1.csv


In [120]:
# Especifica la ruta donde deseas guardar el archivo CSV
ruta_archivo_csv4 = '/Users/pablouriarte/Documents/1. Expediente Tec de Monterrey/1.Tesis/Mapa_Migracion_Irregular_Mexico/3. mapas/1. Mapas/Metodo_1/ciudades_rutas/ciudades detenciones/grid.csv'

# Guardar el DataFrame en el archivo CSV especificado
grid.to_csv(ruta_archivo_csv4, index=False)

print(f"El DataFrame ha sido guardado exitosamente en: {ruta_archivo_csv3}")

El DataFrame ha sido guardado exitosamente en: /Users/pablouriarte/Documents/1. Expediente Tec de Monterrey/1.Tesis/Mapa_Migracion_Irregular_Mexico/3. mapas/1. Mapas/Metodo_1/ciudades_rutas/ciudades detenciones/ciudades_detenciones_1.csv
